<a href="https://colab.research.google.com/github/Abrar-Ahamed/Abu-s_Flutter/blob/main/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0ckeoavl
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0ckeoavl
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=e782f42f60821367cb8c096690e141e9c5f711898a7fa5f20ccda3957f357bff
  Stored in directory: /tmp/pip-ephem-wheel-cache-6o_meb59/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [8]:
%%cu
#include "cuda_runtime.h"

#include "device_launch_parameters.h"

#include "stdio.h"
#include "stdlib.h"
#define COLUMNS 8
#define ROWS 8

_global_ void add(float* image, float* avgimage)
{
    int i = threadIdx.x;
    int j = threadIdx.y;
    printf("\n(%d, %d)", i, j);
    avgimage[i*(ROWS/2)+j]=(image[(2*i)*ROWS+(2*j)]+image[(2*i+1)*ROWS+(2*j)]+image[(2*i)*ROWS+(2*j+1)]+image[(2*i+1)*ROWS+(2*j+1)])/4;
    printf("\navgimage[%d]=(image[%d]+image[%d]+image[%d]+image[%d])/4 = %f",i*(ROWS/2)+j, (2*i)ROWS+(2*j), (2*i+1)*ROWS+(2*j), (2*i)*ROWS+(2*j+1), (2*i+1)*ROWS+(2*j+1), avgimage[i(ROWS/2)+j]);
}

int main()
{
    float image[ROWS][COLUMNS], avgimage[ROWS/2][COLUMNS/2];
    float *cudaImage, *cudaAvgImage;

    cudaMalloc((void **) &cudaImage, ROWS*COLUMNS*sizeof(float));
    cudaMalloc((void *) &cudaAvgImage, (ROWS/2)(COLUMNS/2)*sizeof(float));
    printf("\n\nGenereated image:\n");
    for (int i = 0; i < ROWS; i++)
    {
        for (int j = 0; j < COLUMNS; j++)
        {
            image[i][j]=((rand() % 10000) / 10000.0);
            printf("%f ", image[i][j]);
        }
        printf("\n");
    } 
    
    cudaMemcpy(cudaImage, image, ROWS*COLUMNS*sizeof(int),cudaMemcpyHostToDevice); 
    int numBlocks = 1;
    dim3 threadsPerBlock(ROWS/2, COLUMNS/2);
    add<<<numBlocks,threadsPerBlock>>>(cudaImage, cudaAvgImage);
    cudaMemcpy(avgimage, cudaAvgImage, (ROWS/2)*(COLUMNS/2)*sizeof(float), cudaMemcpyDeviceToHost);
 
    printf("\n\nAveraged image:\n");
    for (int i = 0; i < ROWS/2; i++) // Output Arrays
    {
    for (int j = 0; j < COLUMNS/2; j++)
    {
    printf("%f ",i,j,avgimage[i][j]);
    }
    printf("\n");
    }
    return 0;
}

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(10): error: this declaration has no storage class or type specifier

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(10): error: expected a ";"

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(41): warning: parsing restarts here after previous syntax error

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(43): error: this declaration has no storage class or type specifier

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(43): error: declaration is incompatible with "int printf(const char *, ...)"
/usr/local/cuda/bin/../targets/x86_64-linux/include/crt/common_functions.h(153): here

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(43): error: a value of type "const char *" cannot be used to initialize an entity of type "int"

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(44): error: expected a declaration

/tmp/tmpmlrosrl2/9f7cc030-71ab-46a7-b956-88fff9206cc3.cu(52): warning: pa